In [ ]:
import time
import datetime
import json
import requests
import numpy as np
from matplotlib import pyplot as plt

backend = "cryo-daqbuf"


BASE = "https://data-api.psi.ch"
tzutc = datetime.timezone.utc


def tsfmt(ts):
    return ts.strftime("%Y-%m-%dT%H:%M:%S.%fZ")


def search_channels(name):
    res = requests.get(f"{BASE}/api/4/search/channel?nameRegex={name}")
    if res.status_code != 200:
        raise RuntimeError(f"channel search error {res.status_code}")
    js = res.json()
    return js


def jspr(js):
    jspretty(jscap(4, js))

    
def jspretty(js):
    print(json.dumps(js, indent=2))

    
def jscap(n, js):
    if type(js) == list:
        js = js[:n]
        for i in range(len(js)):
            js[i] = jscap(n, js[i])
    elif type(js) == dict:
        for k in js.keys():
            js[k] = jscap(n, js[k])
    return js


# Convert from daqbuf API data (anchor, ms, ns) to numpy datetime objects.
# Currently, resolution only up to ms.
def to_np_ms(ts_anchor, ts_ms, ts_ns):
    ts_ms = np.array(ts_ms, dtype=np.uint64)
    ts_ns = np.array(ts_ns, dtype=np.uint64)
    ts_abs_ms = ts_ms + 1000 * ts_anchor
    return np.array(ts_abs_ms, dtype="datetime64[ms]")


def unpack_events(js):
    ts_anchor = int(js["tsAnchor"])
    ts_ms = np.array(js["tsMs"], dtype=np.uint64)
    ts_ns = np.array(js["tsNs"], dtype=np.uint64)
    ts = to_np_ms(ts_anchor, ts_ms, ts_ns)
    vals = np.array(js["values"])
    return ts, vals


def fetch_events_js(backend, series, tss):
    ts_perf_1 = time.time()
    ts1 = tss[0]
    ts2 = tss[1]
    tss = list(map(tsfmt, [ts1, ts2]))
    url = "".join([
        f"{BASE}/api/4/events",
        f"?backend={backend}",
        f"&seriesId={series}",
        f"&begDate={tss[0]}",
        f"&endDate={tss[1]}",
    ])
    print(url)
    res = requests.get(url)
    if res.status_code != 200:
        print(res.status_code)
        print(res.content)
        raise RuntimeError(f"can not get event data for series {series} {res.status_code} {res.content.decode()}")
    print(f"Response content size: {len(res.content)//1024} kB")
    if len(res.content) > 1024 * 1024 * 20:
        raise RuntimeError(f"Event data size above threshold")
    ts_perf_2 = time.time()
    print(f"Data request took {(ts_perf_2 - ts_perf_1)*1e3:.0f} ms")
    js = res.json()
    event_count = len(js["tsMs"])
    print(f"Event count {event_count}")
    return js


def plot_events(ts, vals, show_ivl=False):
    nplots = 1
    if show_ivl:
        nplots += 1
    fig, axs = plt.subplots(nplots)
    if nplots == 1:
        axs = [axs]
        fig.set_size_inches((15, 4))
    else:
        fig.set_size_inches((15, 8))
    ax = axs[0]
    ax.plot(ts, vals, '.');
    ax.set_xlabel("Timestamp (UTC)")
    ax.set_ylabel("Value")
    if show_ivl:
        # dt EMA
        ema = 0.0
        ts1 = 0.0
        tsa = np.array(ts, dtype=np.float64)
        if len(tsa) > 0:
            ts1 = tsa[0]
        ys = np.array(tsa)
        for i1, ts2 in enumerate(tsa):
            dt = ts2 - ts1
            ema = ema + 0.08 * (dt - ema)
            ys[i1] = ema
            ts1 = ts2
        ax = axs[1]
        ax.plot(ts, ys, ".")
        ax.set_xlabel("Timestamp (UTC)")
        ax.set_ylabel("Interval EMA (ms)")

        
def fetch_and_plot_events(backend, series, tss, one_before=False, show_ivl=False, show_msp=False):
    js = fetch_events_js(backend, series, tss)
    ts, vals = unpack_events(js)
    plot_events(ts, vals, show_ivl)
    return js,


def ts_dot_series(ts1, ts2):
    a = np.array(ts1)
    return np.append(a, ts2[-1:])


def extend_last(a):
    a = np.array(a)
    return np.append(a, a[-1:])


def unpack_binned(js):
    ts1 = to_np_ms(js["tsAnchor"], js["ts1Ms"], js["ts1Ns"])
    ts2 = to_np_ms(js["tsAnchor"], js["ts2Ms"], js["ts2Ns"])
    counts = np.array(js["counts"])
    mins = np.array(js["mins"])
    maxs = np.array(js["maxs"])
    avgs = np.array(js["avgs"])
    return ts1, ts2, counts, mins, maxs, avgs


def plot_binned(ts1, ts2, counts, mins, maxs, avgs, ylim=None):
    fig, axs = plt.subplots(2)
    fig.set_size_inches((15, 8))
    ax = axs[0]
    ax.bar(x=ts1, width=ts2-ts1, height=maxs-mins, bottom=mins, align="edge", color="tab:blue", alpha=0.4)
    ax.plot(ts_dot_series(ts1, ts2), extend_last(avgs), drawstyle="steps-post", fillstyle="full", linewidth=1.0, color="tab:blue", alpha=0.6)
    ax.hlines(avgs, ts1, ts2, linewidth=4.0, color="tab:blue", alpha=1.0)

    ax = axs[1]
    ax.plot(ts_dot_series(ts1, ts2), extend_last(counts), drawstyle="steps-post")
    lim1 = ts1[0]
    lim2 = ts2[-1]
    limd = lim2 - lim1

    for ax in axs:
        ax.set_xlim((lim1 - 0.05 * limd, lim2 + 0.05 * limd))
    if ylim is not None:
        axs[0].set_ylim(ylim)
    plt.show()


def fetch_binned_js(backend, series, tss, bin_count):
    ts_perf_1 = time.time()
    tsss = list(map(tsfmt, tss))
    url = "".join([
        f"{BASE}/api/4/binned",
        f"?backend={backend}",
        f"&seriesId={series}",
        f"&begDate={tsss[0]}",
        f"&endDate={tsss[1]}",
        f"&binCount={bin_count}",
    ])
    print(url)
    res = requests.get(url)
    if res.status_code != 200:
        print(res.status_code)
        print(res.content)
        raise RuntimeError(f"can not get binned data for series {series}")
    print(f"Response content size: {len(res.content)//1024} kB")
    if len(res.content) > 1024 * 256:
        raise RuntimeError(f"data size above threshold")
    ts_perf_2 = time.time()
    print(f"Data request took {(ts_perf_2 - ts_perf_1) * 1e3 : .0f} ms")
    js = res.json()
    print("Bin count %d" % len(js["ts1Ms"]))
    return js


def fetch_and_plot_binned(backend, series, tss, bin_count, **kargs):
    js = fetch_binned_js(backend, series, tss, bin_count)
    k = unpack_binned(js)
    plot_binned(*k, ylim=kargs.get("ylim"))
    return js,

In [ ]:
js = search_channels(f"PI")
jspr(js)
series = None
a = js["channels"]
if len(a) > 0:
    series = a[0]["seriesId"]
    print(f"Set series: {series}")

In [ ]:
# Select a range around the current time:
tsnow = datetime.datetime.fromtimestamp(time.time(), tz=tzutc)
tss = [
    tsnow - datetime.timedelta(minutes = 60 * 12),
    tsnow,
]

# Can also specify a fixed time window:
_tss = [
    datetime.datetime(2022, 11, 1, tzinfo=tzutc),
    datetime.datetime(2022, 12, 19, tzinfo=tzutc),
]

In [ ]:
js, = fetch_and_plot_events(backend, series, tss)
#jspr(js)

In [ ]:
js, = fetch_and_plot_binned(backend, series, tss, 100)
#jspr(js)

In [ ]:
# Explicit limits on Y-axis:
js, = fetch_and_plot_binned(backend, series, tss, 100, ylim=[1.45, 2.05])